In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import cartopy
import cartopy.crs as ccrs
import pandas as pd
from __future__ import print_function
from datetime import datetime
import scipy
import netCDF4
import time
from minisom import MiniSom

# %pylab inline
# !wget https://www.ncei.noaa.gov/data/international-best-track-archive-for-climate-stewardship-ibtracs/v04r00/access/netcdf/IBTrACS.WP.v04r00.nc

In [ ]:
ibtracs_data = xr.open_dataset('IBTrACS.WP.v04r00.nc')
# select storms over 2010 - 2019
storm_id = ibtracs_data.storm[(ibtracs_data.time[:,0].dt.year > 2009)&(ibtracs_data.time[:,0].dt.year < 2020)]
this_storm = ibtracs_data.sel(storm=storm_id)
storm_time = this_storm.time
this_storm.head()

In [ ]:
this_storm.lat

In [ ]:
# get x,y index (lon/lat of the storms at a moment)
lon=np.trunc(this_storm.lon)
xind=lon

lat=np.trunc(this_storm.lat)
yind=np.zeros_like(lat)
yind=90-lat

In [ ]:
# get the year,month,date,time of storms
stryear = np.empty([309,360],dtype='U64')
strmonth = np.empty([309,360],dtype='U64')
strdate = np.empty([309,360],dtype='U64')
strtime = np.empty([309,360],dtype='U64')
floatwind = np.empty([309,360],dtype='float')
floatpres = np.empty([309,360],dtype='float')
floatdir = np.empty([309,360],dtype='float')
floatlat = np.empty([309,360],dtype='float')
floatlon = np.empty([309,360],dtype='float')
for i in storm_time.storm.data:
    for j in storm_time.date_time.data:
        stryear[i][j]=str(storm_time.values[i][j])[0:4]
        strmonth[i][j]=str(storm_time.values[i][j])[5:7]
        strdate[i][j]=str(storm_time.values[i][j])[8:10]
        strtime[i][j]=str(storm_time.values[i][j])[11:13]
        floatwind[i][j]=this_storm.usa_wind[i][j]
        floatpres[i][j]=this_storm.usa_pres[i][j]
        floatdir[i][j]=this_storm.storm_dir[i][j]
        floatlat[i][j]=this_storm.lat[i][j].values
        floatlon[i][j]=this_storm.lon[i][j].values

In [ ]:
import time
start=time.time()
ds_lst = []
# Read in the files when storms happens
path = '/data/gpm/a/snesbitt/FNL/'
for i in storm_time.storm.data:         # loop over all storm time
    for j in storm_time.date_time.data:         # loop over all time during the storm
        year = stryear[i][j]   # for opening the dataset
        month = strmonth[i][j] # for opening the dataset
        date = strdate[i][j]   # for opening the dataset
        time1 = strtime[i][j]   # for opening the dataset
        if time1=='00' or time1=='06' or time1=='12' or time1=='18':
            filename = 'fnl_'+year+month+date+'_'+time1+'_00'+'.grib2'
            try:
                data1=xr.open_dataset(path+year+'/'+year+'.'+month+'/'+filename,engine='cfgrib',backend_kwargs={'filter_by_keys':{'typeOfLevel': 'isobaricInhPa'}})
                # The center of the certain storm can be found by xind[i][j], yind[i][j]
                a=int(xind[i][j].data) # longitude of typhoon center as the index in this dataset
                b=int(yind[i][j].data) # latitude of typhoon center as the index in this dataset
                # 'data'
                data = xr.Dataset(coords=dict(time=data1['time']),attrs=dict(description='lat from high to low, lon from low to high'))

                data['u_200'] = ['lat','lon'],data1.u.sel(isobaricInhPa=200).isel(longitude=slice(a-30,a+31),latitude=slice(b-30,b+31)).values
                data['v_200'] = ['lat','lon'],data1.v.sel(isobaricInhPa=200).isel(longitude=slice(a-30,a+31),latitude=slice(b-30,b+31)).values
                data['u_850'] = ['lat','lon'],data1.u.sel(isobaricInhPa=850).isel(longitude=slice(a-30,a+31),latitude=slice(b-30,b+31)).values
                data['v_850'] = ['lat','lon'],data1.v.sel(isobaricInhPa=850).isel(longitude=slice(a-30,a+31),latitude=slice(b-30,b+31)).values
                data['gh_200'] = ['lat','lon'],data1.gh.sel(isobaricInhPa=200).isel(longitude=slice(a-30,a+31),latitude=slice(b-30,b+31)).values
                data['gh_500'] = ['lat','lon'],data1.gh.sel(isobaricInhPa=500).isel(longitude=slice(a-30,a+31),latitude=slice(b-30,b+31)).values

                data['usa_wind'] = floatwind[i][j]
                data['usa_pres'] = floatpres[i][j]
                data['storm_dir'] = floatdir[i][j]
                data['latitude'] = floatlat[i][j]
                data['longitude'] = floatlon[i][j]
                
                ds_lst.append(data)
                
#                 if i==0 and j==0:
#                     print('aaaaa')
#                     data_out1=data
#                 else:
#                     print('bbbbb')
#                     data_out1=xr.concat([data_out,data],dim='time')

                data1.close()
            except:
                print(year+month+date+time1)
            
data_out=xr.concat(ds_lst,dim='time')

data_out.to_netcdf('processed_data1.nc')

print(time.time()-start)

In [ ]:
data_out=xr.concat(ds_lst,dim='time')
data_out

In [ ]:
# i=0
# while i < len(ds_lst):
#     if np.size(ds_lst[i].lat)!=61 or np.size(ds_lst[i].lon)!=61:
#         print(ds_lst.pop(i))
#     i+=1
# data_out=xr.concat(ds_lst,dim='time')

In [ ]:
# 2012-08-12T06:00:00.000000000

In [ ]:
data_out.to_netcdf('processed_data1.nc')

In [ ]:
xr.open_dataset('processed_data1.nc')

In [ ]:
m=data1.u.sel(isobaricInhPa=200).isel(longitude=slice(a-30,a+31),latitude=slice(b-30,b+31))-data1.u.sel(isobaricInhPa=850).isel(longitude=slice(a-30,a+31),latitude=slice(b-30,b+31))
m.latitude

In [ ]:
plt.contourf(m.longitude,m.latitude,m)
plt.colorbar()

In [ ]:
plt.contourf(np.flip(data.u_200-data.u_850,0))

In [ ]:
era5 = xr.open_dataset('/data/keeling/a/jye18/c/monsoon_home/ml_597/data_dir/u_850_200_era5.nc',engine='netcdf4')
era5

In [ ]:
# this_storm.storm_dir.expand_dims('time')

In [ ]:
# create 1-d array
storm_dir_reshape = xr.DataArray(np.reshape(np.array(this_storm.storm_dir),(1,-1))[0],dims='time')[::2] 

storm_dir = storm_dir_reshape.to_dataset(name='storm_dir') # convert to xr.DataSet
storm_dir = storm_dir.assign_coords(time=this_storm.storm_dir.time.values.flatten()[::2]) # assign coordinate of time
storm_dir = storm_dir.dropna(dim='time') # drop NaN
storm_dir

In [ ]:
xr.merge([data_out.reset_coords(['step','valid_time'],drop=True),storm_dir],join='outer')

In [ ]:
data_out=xr.open_dataset('/data/gpm/a/fangyiz/processed_data.nc')
data_out

In [ ]:
data_out = data_out.reset_coords(['valid_time','step'],drop=True)

In [ ]:
# create 1-d array
storm_dir_reshape = xr.DataArray(np.reshape(np.array(this_storm.storm_dir),(1,-1))[0],dims='time')[::2] 

storm_dir = storm_dir_reshape.to_dataset(name='storm_dir') # convert to xr.DataSet
storm_dir = storm_dir.assign_coords(time=this_storm.storm_dir.time.values.flatten()[::2]) # assign coordinate of time
storm_dir = storm_dir.dropna(dim='time') # drop NaN
storm_dir['u_200']=np.nan
storm_dir['v_200']=np.nan
storm_dir['u_850']=np.nan
storm_dir['v_850']=np.nan
storm_dir['gh_200']=np.nan
storm_dir['gh_500']=np.nan

In [ ]:
storm_dir

In [ ]:
concat = xr.concat([data_out, storm_dir],dim='time',join='outer')
concat

In [ ]:
# ds_lst.pop(1976)

In [ ]:
# rm /data/gpm/a/snesbitt/FNL/2019/2019.06/fnl_20190624_12_00.grib2
# rm 2011-09-09T06:00:00
# rm 2018-09-07T06:00:00

In [ ]:
som = MiniSom(2, 3, 3721, sigma=1.,
              learning_rate=0.2, neighborhood_function='bubble')  # 3x3 = 9 final colors
som.random_weights_init(height_som)
starting_weights = som.get_weights().copy()  # saving the starting weights
som.train(height_som, 10000, random_order=True)#, verbose=True)

In [ ]:
print('quantization...')
qnt = som.quantization(height_som)  # quantize each pixels of the image
print('building new image...')
clustered = np.zeros((51,100))
qnt
# for i, q in enumerate(qnt):  # place the quantized values into a new image
#     clustered[np.unravel_index(i, shape=(51, 100))] = q
# print('done.')

In [ ]:
plt.subplot(121)
plt.contourf(height_som)
plt.subplot(122)
plt.contourf(qnt)
plt.colorbar()

In [ ]:
import xarray as xr
url = "https://rda.ucar.edu/data/ds633.0/e5.oper.an.pl/201601/e5.oper.an.pl.128_060_pv.ll025sc.2016010100_2016010123.nc"
ds = xr.open_dataset(url)

In [ ]:
from siphon.catalog import TDSCatalog
from siphon.http_util import session_manager

# Set options for Siphon's HTTP session manager--in this case user/password
session_manager.set_session_options(auth=('fangyiz@illinois.edu', 'zfy990714'))
cat = TDSCatalog('https://rda.ucar.edu/thredds/catalog/files/g/ds084.1/2020/20200101/catalog.xml')

selected_dataset = cat.datasets[0]
ds = selected_dataset.remote_access(service='CDMRemote', use_xarray=True)

In [ ]:
import siphon

In [ ]:
data1